In [ ]:
from os.path import expanduser

In [ ]:
from pygccxml import declarations
from pygccxml import utils
from pygccxml import parser

In [ ]:
# Find out the c++ parser. This should resolve to the castxml
# version installed in Docker.
generator_path, generator_name = utils.find_xml_generator()

castxml_git = expanduser("~/devel/CastXML")

castxml_files = [
    f"{castxml_git}/src/Output.h",
    f"{castxml_git}/src/Output.cxx",
]

# Configure the xml generator
config = parser.xml_generator_configuration_t(
    xml_generator_path=generator_path,
    xml_generator=generator_name,
    include_paths=[
        "/usr/lib/llvm-9/include",
        f"{castxml_git}/src",
    ],
    keep_xml = True,
    # TODO(eric.cousineau): This doesn't work?
    flags=["-Wno-unused-value"],
)

In [ ]:
decls = parser.parse(
    castxml_files,
    config,
    compilation_mode=parser.COMPILATION_MODE.ALL_AT_ONCE,
)

In [ ]:
(global_ns,) = decls

In [ ]:
clang = global_ns.namespace("clang")

In [ ]:
len(clang.classes())

In [ ]:
cls = clang.class_("Decl")

In [ ]:
# WARNING: This is slowwww...
global_ns.class_("::clang::ImportDecl")

In [ ]:
declarations.print_declarations([cls])

In [ ]:
for i, cls in enumerate(clang.classes()):
    if i > 100:
        break
    print(cls.name)

In [ ]:
declarations.print_declarations([ns])